In [1]:
%load_ext jupyter_spaces
from IPython.display import display
import numpy as np

In [13]:
import sympy as sym
from sympy import symbols, simplify
from sympy.vector import CoordSys3D, matrix_to_vector

S = CoordSys3D('S')

M, tet1, tet2 = symbols("M theta1 theta2", real=True)
J, mu0, g, t, A = symbols("J mu0 gamma t A", real=True, positive=True)
H = symbols("H", real=True)
#mb should be complex
mx1, my1, mz1 = symbols("m_x1 m_y1 m_z1", real=False)
mx2, my2, mz2 = symbols("m_x2 m_y2 m_z2", real=False)

#define magnetization
m1 = mx1*S.i + my1*S.j + mz1*S.k
m2 = mx2*S.i + my2*S.j + mz2*S.k

#static solution for symmetrical system t1=t2=t
M1_S = M*sym.cos(tet1)*S.i + M*sym.sin(tet1)*S.k
M2_S = M*sym.cos(-tet1)*S.i + M*sym.sin(-tet1)*S.k

M1 = M1_S + m1
M2 = M2_S + m2


#define all the components of Heff
HZ = H*S.k

HD1_S = M1_S.to_matrix(S)[2]*S.k
HD1 = M1.to_matrix(S)[2]*S.k
HD2_S = M2_S.to_matrix(S)[2]*S.k
HD2 = M2.to_matrix(S)[2]*S.k

Hiec1_S = A*(M1_S*(M1_S.dot(M2_S)/(M**2)) - M2_S)
Hiec1 = A*(M1*(M1.dot(M2)/(M**2)) - M2)
Hiec2_S = A*(M2_S*(M2_S.dot(M1_S)/(M**2)) - M1_S)
Hiec2 = A*(M2*(M2.dot(M1)/(M**2)) - M1)

#Hiec1_S = A*(-M2_S)
#Hiec1 = A*(-M2)
#Hiec2_S = A*(-M1_S)
#Hiec2 = A*(-M1)

Heff1_S = HZ + HD1_S + Hiec1_S
Heff1 = HZ + HD1 + Hiec1
Heff2_S = HZ + HD2_S + Hiec2_S
Heff2 = HZ + HD2 + Hiec2

display(simplify(M1.cross(Heff1) - M1_S.cross(Heff1_S)))

w = symbols("w", real=True)
jw = sym.I*w

dm1dt = jw*m1
dm2dt = jw*m2

eq1_S = g*mu0*(M1_S.cross(Heff1_S))
eq1 = dm1dt + g*mu0*(M1.cross(Heff1))
eq2_S = g*mu0*(M2_S.cross(Heff2_S))
eq2 = dm2dt + g*mu0*(M2.cross(Heff2))

eq_mat_S = eq2_S.to_matrix(S).row_insert(0, eq1_S.to_matrix(S))
eq_mat = eq2.to_matrix(S).row_insert(0, eq1.to_matrix(S))

eq_mat_D = ((eq_mat-eq_mat_S)/(g*mu0)).expand()


to_comb_tmp = [mx1, my1, mz1, mx2, my2, mz2]
for i in range(6):
    for j in range(6):
        eq_mat_D = eq_mat_D.subs({to_comb_tmp[i]*to_comb_tmp[j]: 0})
    
eq_mat_D = sym.Matrix(eq_mat_D)
for i in range(6):
    eq_mat_D[i] = eq_mat_D[i].collect((mx1, my1, mz1, mx2, my2, mz2))

eq_mat_D

(A*M*m_y1*sin(theta1) + A*M*m_y2*sin(theta1) - A*m_y1*m_z2 + A*m_y2*m_z1 + H*m_y1 + M*m_y1*sin(theta1) + m_y1*m_z1)*S.i + (-A*M*m_x1*sin(theta1) - A*M*m_x2*sin(theta1) - A*M*m_z1*cos(theta1) + A*M*m_z2*cos(theta1) + A*m_x1*m_z2 - A*m_x2*m_z1 - H*m_x1 - M*m_x1*sin(theta1) - M*m_z1*cos(theta1) - m_x1*m_z1)*S.j + (A*(M*m_y1*cos(theta1) - M*m_y2*cos(theta1) - m_x1*m_y2 + m_x2*m_y1))*S.k

Matrix([
[                                                                   A*M*m_y2*sin(theta1) + m_y1*(A*M*sin(theta1) + H + M*sin(theta1)) + I*m_x1*w/(gamma*mu0)],
[-A*M*m_x2*sin(theta1) + A*M*m_z2*cos(theta1) + m_x1*(-A*M*sin(theta1) - H - M*sin(theta1)) + m_z1*(-A*M*cos(theta1) - M*cos(theta1)) + I*m_y1*w/(gamma*mu0)],
[                                                                                         A*M*m_y1*cos(theta1) - A*M*m_y2*cos(theta1) + I*m_z1*w/(gamma*mu0)],
[                                                                 -A*M*m_y1*sin(theta1) + m_y2*(-A*M*sin(theta1) + H - M*sin(theta1)) + I*m_x2*w/(gamma*mu0)],
[  A*M*m_x1*sin(theta1) + A*M*m_z1*cos(theta1) + m_x2*(A*M*sin(theta1) - H + M*sin(theta1)) + m_z2*(-A*M*cos(theta1) - M*cos(theta1)) + I*m_y2*w/(gamma*mu0)],
[                                                                                        -A*M*m_y1*cos(theta1) + A*M*m_y2*cos(theta1) + I*m_z2*w/(gamma*mu0)]])

In [4]:
lin_mat = sym.Matrix()

to_comb_tmp = [mx1, my1, mz1, mx2, my2, mz2]

for i in range(6):
    col = []
    for j in range(6):
         col.append(eq_mat_D[j].coeff(to_comb_tmp[i], 1))
    lin_mat = lin_mat.col_insert(i,sym.Matrix(col))

lin_mat = simplify(lin_mat)
lin_mat

lin_mat_det = sym.trigsimp(lin_mat.det().collect(w))
lin_mat_det

(gamma**4*mu0**4*w**2*(8*A**3*M**4*sin(theta1)**4 - 8*A**3*M**4*sin(theta1)**2 + 4*A**2*H**2*M**2*sin(theta1)**2 - 4*A**2*H**2*M**2 + 4*A**2*M**4*sin(theta1)**4 - 8*A**2*M**4*sin(theta1)**2 + 6*A*H**2*M**2*sin(theta1)**2 - 2*A*H**2*M**2 - 2*A*M**4*sin(theta1)**4 - 2*A*M**4*sin(theta1)**2 - H**4 + 2*H**2*M**2*sin(theta1)**2 - M**4*sin(theta1)**4) + 2*gamma**2*mu0**2*w**4*(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2) - w**6)/(gamma**6*mu0**6)

In [5]:
lin_mat_det0_sol = sym.solve(lin_mat_det, w,
          simplify=False, manual=False, implicit=False, 
          warn=False, check=True, exclude=[H, tet1, tet2, A, M])

lin_mat_det0_sol

[0,
 -sqrt(2*A**2*M**2*gamma**2*mu0**2*cos(theta1)**2 - A*M**2*gamma**2*mu0**2*cos(theta1)**2 + 2*A*M**2*gamma**2*mu0**2 + H**2*gamma**2*mu0**2 - M**2*gamma**2*mu0**2*cos(theta1)**2 + M**2*gamma**2*mu0**2 - M*gamma**2*mu0**2*sqrt(2*A + 1)*sqrt(2*A**3*M**2*cos(theta1)**4 + 4*A**2*M**2*sin(theta1)**4 - 4*A**2*M**2*sin(theta1)**2 - 3*A**2*M**2*cos(theta1)**4 + 4*A**2*M**2*cos(theta1)**2 + 2*A*H**2*sin(theta1)**2 + 2*A*H**2*cos(theta1)**2 - 2*A*H**2 - 2*A*M**2*sin(theta1)**2 - 2*A*M**2*cos(theta1)**2 + 2*A*M**2 + 2*H**2*sin(theta1)**2 - 2*H**2*cos(theta1)**2 + 2*H**2 - M**2*sin(theta1)**4 + M**2*cos(theta1)**4 - 2*M**2*cos(theta1)**2 + M**2)),
 sqrt(2*A**2*M**2*gamma**2*mu0**2*cos(theta1)**2 - A*M**2*gamma**2*mu0**2*cos(theta1)**2 + 2*A*M**2*gamma**2*mu0**2 + H**2*gamma**2*mu0**2 - M**2*gamma**2*mu0**2*cos(theta1)**2 + M**2*gamma**2*mu0**2 - M*gamma**2*mu0**2*sqrt(2*A + 1)*sqrt(2*A**3*M**2*cos(theta1)**4 + 4*A**2*M**2*sin(theta1)**4 - 4*A**2*M**2*sin(theta1)**2 - 3*A**2*M**2*cos(theta1)**4

In [6]:
for i in range(len(lin_mat_det0_sol)):
    lin_mat_det0_sol[i] = simplify(simplify(lin_mat_det0_sol[i]))
    display(lin_mat_det0_sol[i])

0

-gamma*mu0*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 - M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

gamma*mu0*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 - M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

-gamma*mu0*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

gamma*mu0*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

In [7]:
#try to solve with linsolve
lin_mat_sol = sym.solve_linear_system(lin_mat.col_insert(6, sym.Matrix([0]*6)).subs({w: lin_mat_det0_sol[2]}), mx1, my1, mz1, mx2, my2, mz2, 
                        simplify=False, manual=True, implicit=False, 
                        warn=True, check=False, exclude=[H, tet1, tet2, A, M, w])

lin_mat_sol

KeyboardInterrupt: 

In [7]:
#prepare system for solve
system = []

to_comb_tmp = [mx1, my1, mz1, mx2, my2, mz2]
for i in range(6):
    cur_eq = sym.Rational(0)
    for j in range(6):
        cur_eq += lin_mat.row(i)[j] * to_comb_tmp[j]
    system.append(simplify(cur_eq.subs({w: lin_mat_det0_sol[4]})))

for i in range(6):
    display(system[i])
    


A*M*m_y2*sin(theta1) + I*m_x1*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2)) + m_y1*(A*M*sin(theta1) + H + M*sin(theta1))

-A*M*m_x2*sin(theta1) + A*M*m_z2*cos(theta1) - M*m_z1*(A + 1)*cos(theta1) - m_x1*(A*M*sin(theta1) + H + M*sin(theta1)) + I*m_y1*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

A*M*m_y1*cos(theta1) - A*M*m_y2*cos(theta1) + I*m_z1*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

-A*M*m_y1*sin(theta1) + I*m_x2*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2)) - m_y2*(A*M*sin(theta1) - H + M*sin(theta1))

A*M*m_x1*sin(theta1) + A*M*m_z1*cos(theta1) - M*m_z2*(A + 1)*cos(theta1) + m_x2*(A*M*sin(theta1) - H + M*sin(theta1)) + I*m_y2*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

-A*M*m_y1*cos(theta1) + A*M*m_y2*cos(theta1) + I*m_z2*sqrt(2*A**2*M**2*cos(theta1)**2 - A*M**2*cos(theta1)**2 + 2*A*M**2 + H**2 - M**2*cos(theta1)**2 + M**2 + M*sqrt(2*A + 1)*sqrt(2*A**3*M**2*sin(theta1)**4 - 4*A**3*M**2*sin(theta1)**2 + 2*A**3*M**2 + A**2*M**2*sin(theta1)**4 - 2*A**2*M**2*sin(theta1)**2 + A**2*M**2 + 4*H**2*sin(theta1)**2))

In [23]:
#try to solve with solve
#replace first argument with system
lin_mat_sol2 = sym.solve(eq_mat_D.subs({w: lin_mat_det0_sol[3]}), mx1, my1, mz1, mx2, my2, mz2,
                        simplify=False, manual=True, implicit=True, 
                        warn=True, check=True)

lin_mat_sol2

[]

In [1]:
#numerical frequency calculation
mu0_num = 1.2566370621219e-6
g_num = 1.76e11
J_num = 2.7e-3
t_num = 5e-9
M_num = 1042e3
H_num = 110e-3/mu0_num
A_num = J_num/(mu0_num*t*(M_num**2))
cos_tet_num = mu0_num*t_num*H_num*M_num/(2*J_num+mu0_num*t_num*(M_num**2))

for cur_sol in lin_mat_det0_sol:
    freq_eval = cur_sol.subs({
        mu0: mu0_num,
        g: g_num,
        J: J_num,
        t: t_num,
        M: M_num,
        H: H_num,
        A: A_num,
        sym.sin(tet1)**2: 1-sym.cos(tet1)**2
    }).subs({
        sym.cos(tet1): cos_tet_num
    })
    display((freq_eval.evalf()/(2*np.pi))*1e-9)

display("default freq:")
display((g_num*mu0_num*np.sqrt(H_num*(H_num+M_num))/(2*np.pi))*1e-9)

NameError: name 't' is not defined

In [55]:
lin_mat_eval = lin_mat.subs({
   w: lin_mat_det0_sol[2]
}).subs({
    mu0: mu0_num,
    g: g_num,
    J: J_num,
    t: t_num,
    M: M_num,
    H: H_num,
    A: A_num,
    sym.cos(tet1): cos_tet_num,
    sym.sin(tet1): np.sqrt(1-cos_tet_num**2.0)
}).evalf()

lin_mat_eval = np.array((lin_mat_eval)).astype(np.complex)
#np.linalg.solve(lin_mat_eval,np.array([0.0]*6))
svd_res = np.linalg.svd(lin_mat_eval)
np.angle(svd_res[2][-1])

array([ 3.14159265e+00,  1.57079633e+00, -5.94459364e-15, -4.01527239e-16,
       -1.57079633e+00, -3.14159265e+00])

In [47]:
np.abs(eig_vec[4]/eig_vec[4][5])

array([7.63794062e+14, 5.15072263e+15, 4.62999811e+00, 7.63794062e+14,
       5.15072263e+15, 1.00000000e+00])